In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [38]:
block_size = 8
batch_size = 4
max_iters = 10000
lr = 3e-4

In [36]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [29]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100]) 

tensor([28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36, 11,
         1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32, 39,
        42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,
         1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1, 39,
        50,  9,  1, 39, 50, 37, 25,  1, 39, 30])


In [47]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    xi = torch.randint(len(data) - block_size, (batch_size,)) # random starting indices
    # print(f"Training Integers: {xi}")
    # input batch
    x = torch.stack([data[i:i+block_size] for i in xi])
    # target batch    
    y = torch.stack([data[i+1:i+1+block_size] for i in xi])
    x, y = x.to(device), y.to(device) # send to cuda -> parallelize
    return x, y

x, y = get_batch('train')
print("Input batch:")
print(x)
print("Target batch:")
print(y)    

Input batch:
tensor([[65, 64, 72,  1, 57, 68, 67,  5],
        [74,  1, 72, 74, 71, 69, 71, 62],
        [74, 62, 56, 64,  2,  1, 44, 68],
        [67, 57,  1, 73, 68, 65, 57,  1]], device='cuda:0')
Target batch:
tensor([[64, 72,  1, 57, 68, 67,  5, 73],
        [ 1, 72, 74, 71, 69, 71, 62, 72],
        [62, 56, 64,  2,  1, 44, 68,  1],
        [57,  1, 73, 68, 65, 57,  1, 74]], device='cuda:0')


In [40]:
# basic bigram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B=batch size, T=time, C=channels (vocab size)
            
            # reshape logits and targets for cross-entropy loss
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)  # get predictions
            logits = logits[:, -1, :] # (B, C) last time step
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size).to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)        


nuB6cpji4o-P1)
Wv427Gl4osalbYfcv0TadF_odLY(pv,8*h?7vB& R'4eQaZ
9hQgiRv2Fr4tBMT0(NcC2FpT*V9,pT*ZYoIx5KvpNsag6_q.4IZdJAS?QPbSE;rY
dyq.mLyZF-5uK[_DQ6ch?vhkTmv!if!gfU6VR-z;Vrh;G);.JZKJ_D9Jd :eg8
qh44th0J.:1*n!8F-POk[MQw:-Wwx,v&L3!e2d.JK8j(WVfBjfdFvn3KO"fLTpowAb-MQlm3WlC5mI5o&.gQcq.I9Ur)x.opg;D
hX92pT-R_x*kkmp?y-e19Oq.8J:3Cr4ISYvz2Wv0XSh"xsYSjy0vUy
zwxyvLYaF-e?E?MQK"cSCVf ]]9JMyVn3QkkT.CKpXaI8Te6tIFAJv-JvZ[Zr[-)7GFu)_k7;8-3cIxY pK))P--xODvgs2SoC?-Dv;X1sX7cJdk5xn3 m[v?_Zx
-dP3ckw(UUJrDQKZF(X94tkRTwP32


In [48]:
optimizer = torch.optim.AdamW(model.parameters(), lr)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.399839162826538


In [49]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars) 


Sbed GVIts,"He bellitess thoff goce whom nimave a 4 on hos f
GAhisthby d vesewheeracasin whene," oul agors itofinade
s jqusag to the we h baro opllore 's sctheman y oom te m ld.
plf, wins andaty nsteredongrthe can  ithe tcabon:k Thins r sthabo!"

t aply ce Bir  Winoutheme te mand anound ckean.
oy s ve, r, bevey f me thallly ck Is lod;y:jN_YN" NIZe This ipsorevoowa  pe s Wed pl fure. bousonchthas at.
nod tl lwale hebsky Ar s. at, s sithe

Dd, LYOhintan re, fixp!"Tumeliole per s Tharouplerd th
THo
